# Проект 4. "Авиарейсы без потерь"

Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Первичный осмотр данных

In [2]:
df = pd.read_csv('flights.csv')

In [3]:
df.head()

,flight_id,flight_no,departure_airport,arrival_airport,arrival_city,actual_departure,actual_arrival,flight_time,model,aircraft_code,...,total_seats,flight_id-2,sold_tickets_economy,sold_tickets_comfort,sold_tickets_bisiness,sold_tickets_total,revenue_economy,revenue_comfort,revenue_bisiness,revenue_total
0,136119,PG0252,AAQ,SVO,Moscow,2017-01-03 10:08:00+00,2017-01-03 11:49:00+00,101,Boeing 737-300,733,...,130,136119.0,102.0,0.0,11.0,113.0,1250400.0,NaN,402600.0,1653000.0
1,136120,PG0252,AAQ,SVO,Moscow,2017-02-24 10:07:00+00,2017-02-24 11:46:00+00,99,Boeing 737-300,733,...,130,136120.0,98.0,0.0,11.0,109.0,1202800.0,NaN,402600.0,1605400.0
2,136122,PG0252,AAQ,SVO,Moscow,2017-01-08 10:08:00+00,2017-01-08 11:48:00+00,100,Boeing 737-300,733,...,130,136122.0,87.0,0.0,10.0,97.0,1065000.0,NaN,366000.0,1431000.0
3,136130,PG0252,AAQ,SVO,Moscow,2017-02-13 10:07:00+00,2017-02-13 11:46:00+00,99,Boeing 737-300,733,...,130,136130.0,97.0,0.0,10.0,107.0,1190600.0,NaN,366000.0,1556600.0
4,136131,PG0252,AAQ,SVO,Moscow,2017-01-28 10:07:00+00,2017-01-28 11:46:00+00,99,Boeing 737-300,733,...,130,136131.0,112.0,0.0,12.0,124.0,1373600.0,NaN,439200.0,1812800.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   flight_id              127 non-null    int64  
 1   flight_no              127 non-null    object 
 2   departure_airport      127 non-null    object 
 3   arrival_airport        127 non-null    object 
 4   arrival_city           127 non-null    object 
 5   actual_departure       127 non-null    object 
 6   actual_arrival         127 non-null    object 
 7   flight_time            127 non-null    int64  
 8   model                  127 non-null    object 
 9   aircraft_code          127 non-null    object 
 10  economy_seats          127 non-null    int64  
 11  comfort_seats          127 non-null    int64  
 12  bisiness_seats         127 non-null    int64  
 13  total_seats            127 non-null    int64  
 14  flight_id-2            118 non-null    float64
 15  sold_t

Очистка и подготовка данных

In [5]:
# информация о местах комфорт класса отсутствует, поэтому удалим столбцы, относящиеся к ним
df.drop('revenue_comfort', axis=1, inplace=True)
df.drop('comfort_seats', axis=1, inplace=True)
df.drop('sold_tickets_comfort', axis=1, inplace=True)

In [6]:
# столбец flight_id-2 удалим как дублирующий значения в столбце flight_id
df.drop('flight_id-2', axis=1, inplace=True)

In [7]:
df.dropna(inplace=True) #удалим рейсы, по которым нет купленных билетов

In [8]:
df.describe()

,flight_id,flight_time,economy_seats,bisiness_seats,total_seats,sold_tickets_economy,sold_tickets_bisiness,sold_tickets_total,revenue_economy,revenue_bisiness,revenue_total
count,118.000000,118.000000,118.000000,118.0,118.000000,118.000000,118.000000,118.000000,1.180000e+02,118.000000,1.180000e+02
mean,136533.440678,74.957627,101.500000,12.0,113.500000,90.864407,10.788136,101.652542,8.796102e+05,296895.762712,1.176506e+06
std,261.971994,24.992783,16.570363,0.0,16.570363,14.028003,1.395057,14.295643,3.877180e+05,96815.426116,4.794251e+05
min,136119.000000,49.000000,85.000000,12.0,97.000000,54.000000,6.000000,64.000000,3.420000e+05,113400.000000,5.310000e+05
25%,136286.250000,50.000000,85.000000,12.0,97.000000,83.000000,10.000000,95.000000,5.259000e+05,226800.000000,7.464000e+05
50%,136528.500000,74.500000,101.500000,12.0,113.500000,85.500000,11.000000,97.000000,7.956500e+05,226800.000000,1.081650e+06
75%,136768.500000,100.000000,118.000000,12.0,130.000000,101.750000,12.000000,111.750000,1.246750e+06,402600.000000,1.625850e+06
max,136956.000000,103.000000,118.000000,12.0,130.000000,118.000000,12.000000,130.000000,1.446800e+06,439200.000000,1.886000e+06


In [9]:
# Можно сразу отметить, что во всех рассматриваемых в дата сете самолетах стандартное число мест бизнес-класса: 12

In [10]:
# Преобразуем для удобства форматы

df.actual_departure = pd.to_datetime(df.actual_departure) # время отлета - во временной формат
df.actual_arrival = pd.to_datetime(df.actual_arrival) # время прилета - во временной формат

# все остальные числовые данные сделаем целочисленными
df.flight_id = df.flight_id.astype('int64')
df.flight_time = df.flight_time.astype('int64')
df.economy_seats = df.economy_seats.astype('int64')
df.bisiness_seats = df.bisiness_seats.astype('int64')
df.total_seats = df.total_seats.astype('int64')
df.sold_tickets_economy = df.sold_tickets_economy.astype('int64')
df.sold_tickets_bisiness = df.sold_tickets_bisiness.astype('int64')
df.sold_tickets_total = df.sold_tickets_total.astype('int64')
df.revenue_economy = df.revenue_economy.astype('int64')
df.revenue_bisiness = df.revenue_bisiness.astype('int64')
df.revenue_total = df.revenue_total.astype('int64')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118 entries, 0 to 117
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   flight_id              118 non-null    int64              
 1   flight_no              118 non-null    object             
 2   departure_airport      118 non-null    object             
 3   arrival_airport        118 non-null    object             
 4   arrival_city           118 non-null    object             
 5   actual_departure       118 non-null    datetime64[ns, UTC]
 6   actual_arrival         118 non-null    datetime64[ns, UTC]
 7   flight_time            118 non-null    int64              
 8   model                  118 non-null    object             
 9   aircraft_code          118 non-null    object             
 10  economy_seats          118 non-null    int64              
 11  bisiness_seats         118 non-null    int64              

Для определения прибыльности/убыточности рейсов в датасете не хватает данных о расходах на перелет.
Есть только данные о полученной выручке (revenue).
В действительности, расходы у авиакомпаний могут сильно различаться с учетом эффекта масштаба операционной деятельности, стоимости лизинга самолетов (а у большинства авиакомпаний самолеты находятся именно в лизинге) и прочих факторов.
В данном кейсе ограничимся расходами на топливо и прочими условно переменными расходами.
По различным источникам доля расходов на топливо у авиакомпаний может варьироваться от 13% до 30% в зависимости от цены на топливо и прочих влияющих факторов.
Чтобы не ограничиваться только расходами на топливо, примем их долю в расходах за 20%.
Данные о стоимости авиационного топлива в течение зимнего периода 2017 года в Анапе возьмем из официальных источников:
https://favt.gov.ru/dejatelnost-ajeroporty-i-ajerodromy-ceny-na-aviagsm/

Создание дополнительных признаков для проведения анализа прибыльности/убыточности авиарейсов

In [12]:
df.flight_time = df.flight_time / 60 # время полета переведем из минут в часы

In [13]:
# Создаем дополнительный столбец с ценами на авиатопливо (в руб/кг без НДС)

# Цены в рублях на авиатопливо в Анапе (1 тн без НДС) согласно официальных источников:
# 41 435 - январь 2017
# 45 510 - ноябрь 2017
# 47 101 - декабрь 2017

df['fuel_price'] = df['actual_departure'].apply(lambda x: 41.435 if x.month == 1 else 45.510 if x.month == 11 else 47.101)

In [20]:
# Создаем столбец с расходами на топливо 
# Количество потребляемого топлива за один час перелета самолетов из дата сета определяем как:
# Boeing 737-300 — 2400 kg/h
# Sukhoi Superjet 100 — 1700 kg/h
# Источник: https://www.airliners.net/forum/viewtopic.php?t=1355819
# Умножаем время перелета в часах на количество потребляемого топлива и его цену в рублях без НДС

df['fuel_consumption_per_km'] = df.model.apply(lambda x: 2400 if x == 'Boeing 737-300' else 1700)
df['fuel_cost'] = df.fuel_consumption_per_km * df.fuel_price * df.flight_time

In [21]:
df

,flight_id,flight_no,departure_airport,arrival_airport,arrival_city,actual_departure,actual_arrival,flight_time,model,aircraft_code,...,sold_tickets_bisiness,sold_tickets_total,revenue_economy,revenue_bisiness,revenue_total,fuel_price,fuel_consumption_per_km,fuel_cost,total_costs,income
0,136119,PG0252,AAQ,SVO,Moscow,2017-01-03 10:08:00+00:00,2017-01-03 11:49:00+00:00,1.683333,Boeing 737-300,733,...,11,113,1250400,402600,1653000,41.435,2400,167397.400000,836987.000000,816013
1,136120,PG0252,AAQ,SVO,Moscow,2017-02-24 10:07:00+00:00,2017-02-24 11:46:00+00:00,1.650000,Boeing 737-300,733,...,11,109,1202800,402600,1605400,47.101,2400,186519.960000,932599.800000,672800
2,136122,PG0252,AAQ,SVO,Moscow,2017-01-08 10:08:00+00:00,2017-01-08 11:48:00+00:00,1.666667,Boeing 737-300,733,...,10,97,1065000,366000,1431000,41.435,2400,165740.000000,828700.000000,602300
3,136130,PG0252,AAQ,SVO,Moscow,2017-02-13 10:07:00+00:00,2017-02-13 11:46:00+00:00,1.650000,Boeing 737-300,733,...,10,107,1190600,366000,1556600,47.101,2400,186519.960000,932599.800000,624000
4,136131,PG0252,AAQ,SVO,Moscow,2017-01-28 10:07:00+00:00,2017-01-28 11:46:00+00:00,1.650000,Boeing 737-300,733,...,12,124,1373600,439200,1812800,41.435,2400,164082.600000,820413.000000,992387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,136936,PG0480,AAQ,EGO,Belgorod,2017-01-02 09:28:00+00:00,2017-01-02 10:18:00+00:00,0.833333,Sukhoi Superjet-100,SU9,...,9,82,462300,170100,632400,41.435,1700,58699.583333,293497.916667,338902
114,136937,PG0480,AAQ,EGO,Belgorod,2017-01-03 09:29:00+00:00,2017-01-03 10:19:00+00:00,0.833333,Sukhoi Superjet-100,SU9,...,9,81,456000,170100,626100,41.435,1700,58699.583333,293497.916667,332602
115,136951,PG0480,AAQ,EGO,Belgorod,2017-02-02 09:28:00+00:00,2017-02-02 10:17:00+00:00,0.816667,Sukhoi Superjet-100,SU9,...,12,90,493800,226800,720600,47.101,1700,65391.888333,326959.441667,393640
116,136953,PG0480,AAQ,EGO,Belgorod,2017-02-27 09:27:00+00:00,2017-02-27 10:17:00+00:00,0.833333,Sukhoi Superjet-100,SU9,...,12,97,538500,226800,765300,47.101,1700,66726.416667,333632.083333,431667


In [22]:
# Суммарные расходы

df['total_costs'] = df.fuel_cost * 5

#  Прибыль / убыток (вычитаем НДС - 18%)

df['income'] = (df.revenue_total - df.total_costs)

In [23]:
df.income = df.income.astype('int64')


In [24]:
df

,flight_id,flight_no,departure_airport,arrival_airport,arrival_city,actual_departure,actual_arrival,flight_time,model,aircraft_code,...,sold_tickets_bisiness,sold_tickets_total,revenue_economy,revenue_bisiness,revenue_total,fuel_price,fuel_consumption_per_km,fuel_cost,total_costs,income
0,136119,PG0252,AAQ,SVO,Moscow,2017-01-03 10:08:00+00:00,2017-01-03 11:49:00+00:00,1.683333,Boeing 737-300,733,...,11,113,1250400,402600,1653000,41.435,2400,167397.400000,836987.000000,816013
1,136120,PG0252,AAQ,SVO,Moscow,2017-02-24 10:07:00+00:00,2017-02-24 11:46:00+00:00,1.650000,Boeing 737-300,733,...,11,109,1202800,402600,1605400,47.101,2400,186519.960000,932599.800000,672800
2,136122,PG0252,AAQ,SVO,Moscow,2017-01-08 10:08:00+00:00,2017-01-08 11:48:00+00:00,1.666667,Boeing 737-300,733,...,10,97,1065000,366000,1431000,41.435,2400,165740.000000,828700.000000,602300
3,136130,PG0252,AAQ,SVO,Moscow,2017-02-13 10:07:00+00:00,2017-02-13 11:46:00+00:00,1.650000,Boeing 737-300,733,...,10,107,1190600,366000,1556600,47.101,2400,186519.960000,932599.800000,624000
4,136131,PG0252,AAQ,SVO,Moscow,2017-01-28 10:07:00+00:00,2017-01-28 11:46:00+00:00,1.650000,Boeing 737-300,733,...,12,124,1373600,439200,1812800,41.435,2400,164082.600000,820413.000000,992387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,136936,PG0480,AAQ,EGO,Belgorod,2017-01-02 09:28:00+00:00,2017-01-02 10:18:00+00:00,0.833333,Sukhoi Superjet-100,SU9,...,9,82,462300,170100,632400,41.435,1700,58699.583333,293497.916667,338902
114,136937,PG0480,AAQ,EGO,Belgorod,2017-01-03 09:29:00+00:00,2017-01-03 10:19:00+00:00,0.833333,Sukhoi Superjet-100,SU9,...,9,81,456000,170100,626100,41.435,1700,58699.583333,293497.916667,332602
115,136951,PG0480,AAQ,EGO,Belgorod,2017-02-02 09:28:00+00:00,2017-02-02 10:17:00+00:00,0.816667,Sukhoi Superjet-100,SU9,...,12,90,493800,226800,720600,47.101,1700,65391.888333,326959.441667,393640
116,136953,PG0480,AAQ,EGO,Belgorod,2017-02-27 09:27:00+00:00,2017-02-27 10:17:00+00:00,0.833333,Sukhoi Superjet-100,SU9,...,12,97,538500,226800,765300,47.101,1700,66726.416667,333632.083333,431667


In [25]:
print(f'Доля убыточных рейсов составляет {round(df[df.income < 0].shape[0] / df.shape[0] * 100, 2)} процентов')

Доля убыточных рейсов составляет 0.0 процентов
